## Ecospold 2 parser

In [42]:
import xml.etree.ElementTree
import xlrd, xlwt
import numpy as np

In [3]:
DataListFile        = 'C:\\Users\\spauliuk\\FILES\\Work_Archive\\_Data_Raw\\ecoinvent\\3.3\\ei_33_rawdata_parse.xlsx'
DataListFileHandle  = xlrd.open_workbook(DataListFile)
DataListFileSheet   = DataListFileHandle.sheet_by_name('ParseList')

FileNames = [] #List of ecospold filenames
m = 0
while True:
    try:
        FileNames.append(DataListFileSheet.cell_value(m +2, 2))
        m += 1
    except:
        break
        
print(FileNames)        

['0a36a668-a8eb-46e8-a342-4a8196d56f73', '39c4aa59-f663-4ea3-aacc-acda2f9e5482', '5476101e-f839-477c-9fb6-5200879563b8', 'c201f0e7-af66-49a3-9ad7-a55172d8aa5a', 'ec37499b-27ba-419a-b3f5-971a0b623069']


In [47]:
# Open Excel for writing
myfont = xlwt.Font()
myfont.bold = True
mystyle = xlwt.XFStyle()
mystyle.font = myfont
Result_workbook  = xlwt.Workbook(encoding = 'ascii') 

for FN in FileNames:
    File = 'C:\\Users\\spauliuk\\FILES\\Work_Archive\\_Data_Raw\\ecoinvent\\3.3\\ei33_unlinked\\' + FN + '.spold'
    e = xml.etree.ElementTree.parse(File).getroot()
    Result_worksheet = Result_workbook.add_sheet(FN[0:15]+ '...') 
    Result_worksheet.write(1, 1, label = 'Activity name', style = mystyle)  
    Result_worksheet.write(1, 2, label = 'Activity ID', style = mystyle)  
    Result_worksheet.write(1, 3, label = 'Activity region', style = mystyle)  
    Result_worksheet.write(1, 4, label = 'Activity start date', style = mystyle)  
    Result_worksheet.write(1, 5, label = 'Activity end date', style = mystyle)  
    Result_worksheet.write(1, 6, label = 'Activity description', style = mystyle)  
    Result_worksheet.write(4, 1, label = 'Flow data', style = mystyle)  
    
    Result_worksheet.write(4, 2, label = 'Good/substance', style = mystyle)  
    Result_worksheet.write(4, 3, label = 'Good/substance ID', style = mystyle)  
    Result_worksheet.write(4, 4, label = 'Amount', style = mystyle)  
    Result_worksheet.write(4, 5, label = 'Unit', style = mystyle)  
    Result_worksheet.write(4, 6, label = 'Uncertainty', style = mystyle)  
    Result_worksheet.write(4, 7, label = 'Group', style = mystyle)  
    Result_worksheet.write(4, 8, label = 'Direction', style = mystyle)  
    Result_worksheet.write(4, 9, label = 'Prive level', style = mystyle)  
    Result_worksheet.write(4, 10, label = 'Price unit', style = mystyle)  
    m0 = 0 # iterate over flow entries
    for child in e[0]: 
        # Check for activity description
        if child.tag == '{http://www.EcoInvent.org/EcoSpold02}activityDescription':
            for childx in child:
                if childx.tag == '{http://www.EcoInvent.org/EcoSpold02}activity':
                    Act_ID = childx.attrib['id']
                    Result_worksheet.write(2, 2, label = Act_ID)                     
                    for childxx in childx:
                        if childxx.tag == '{http://www.EcoInvent.org/EcoSpold02}activityName':                    
                            Act_Name = childxx.text
                            Result_worksheet.write(2, 1, label = Act_Name)     
                        if childxx.tag == '{http://www.EcoInvent.org/EcoSpold02}generalComment':                    
                            for childxx3 in childxx:
                                if childxx3.tag == '{http://www.EcoInvent.org/EcoSpold02}text':
                                    if childxx3.attrib['index'] == '1':
                                        Act_Description = childxx3.text            
                                        Result_worksheet.write(2, 6, label = Act_Description)     
                if childx.tag == '{http://www.EcoInvent.org/EcoSpold02}geography':
                    for childxx2 in childx:
                        if childxx2.tag == '{http://www.EcoInvent.org/EcoSpold02}shortname':                    
                            Act_Region = childxx2.text                            
                            Result_worksheet.write(2, 3, label = Act_Region)    
                if childx.tag == '{http://www.EcoInvent.org/EcoSpold02}timePeriod':
                    Act_startDate = childx.attrib['startDate']
                    Act_endDate   = childx.attrib['endDate']  
                    Result_worksheet.write(2, 4, label = Act_startDate) 
                    Result_worksheet.write(2, 5, label = Act_endDate) 
                    
        # Check for flow data
        if child.tag == '{http://www.EcoInvent.org/EcoSpold02}flowData':
            rowc = 5
            PriceLevel = None
            Uncert     = 'None'
            for childx in e[0][m0]: # Check for flows (exchanges)
                TakeFlag = 0 # must be 1 if this flow is used for export (exlude parameters etc.)
                if 'intermediateExchangeId' in childx.attrib:
                    FlowGroup = 'Intermediate Exchange'
                    ThisID = childx.attrib['id']
                    TakeFlag = 1
                if 'elementaryExchangeId' in childx.attrib:
                    FlowGroup = 'Elementary Exchange'
                    ThisID = childx.attrib['id']
                    TakeFlag = 1
                mx = 0  # iterate over attributes of flow entry
                for childxx in childx: # iterate over all attributes of a given flow
                    #print(childxx.tag)
                    if  childxx.tag == '{http://www.EcoInvent.org/EcoSpold02}name': # name of flowing good/substance
                        FlowName = childxx.text
                    if  childxx.tag == '{http://www.EcoInvent.org/EcoSpold02}unitName': # name of unit
                        UnitName = childxx.text
                    if  childxx.tag == '{http://www.EcoInvent.org/EcoSpold02}inputGroup': # flow is input
                        FlowDirection = 'Input'
                    if  childxx.tag == '{http://www.EcoInvent.org/EcoSpold02}outputGroup': # flow is output
                        FlowDirection = 'Output'
                    if  childxx.tag == '{http://www.EcoInvent.org/EcoSpold02}property': # property info
                        ThisAmount = childxx.attrib['amount']
                        for childxxa in childxx:
                            if childxxa.tag == '{http://www.EcoInvent.org/EcoSpold02}name':
                                ThisName = childxxa.text
                            if childxxa.tag == '{http://www.EcoInvent.org/EcoSpold02}unitName': 
                                ThisUnitName = childxxa.text
                        if ThisName == 'price':
                            PriceLevel = ThisAmount
                            PriceUnit  = ThisUnitName
                    if   childxx.tag == '{http://www.EcoInvent.org/EcoSpold02}uncertainty': # uncertainty info
                        for childxxx in childxx: # children of uncertainty
                            if childxxx.tag == '{http://www.EcoInvent.org/EcoSpold02}lognormal':
                                #print(childxxx.attrib)
                                Uncert = '2;' + str(childxxx.attrib['mu']) + ';' + str(np.sqrt(np.float(childxxx.attrib['varianceWithPedigreeUncertainty']))) + ';none;none;none'

                            #FlowUncertainty = 'Output'                    
                    mx += 1


                if TakeFlag == 1:
                    # print(FlowName,'  ', childx.attrib['amount'],'  ',UnitName,'  ',FlowGroup + ', ' + FlowDirection)  
                    Result_worksheet.write(rowc, 2, label = FlowName)    
                    Result_worksheet.write(rowc, 3, label = ThisID)    
                    Result_worksheet.write(rowc, 4, label = childx.attrib['amount'])    
                    Result_worksheet.write(rowc, 5, label = UnitName)    
                    Result_worksheet.write(rowc, 6, label = Uncert)    
                    Result_worksheet.write(rowc, 7, label = FlowGroup)    
                    Result_worksheet.write(rowc, 8, label = FlowDirection)    
                    if PriceLevel is not None:
                        Result_worksheet.write(rowc, 9, label = PriceLevel) 
                        Result_worksheet.write(rowc, 10, label = PriceUnit) 
                    rowc +=1
                
                PriceLevel = None

        m0 += 1    

Result_workbook.save('C:\\Users\\spauliuk\\FILES\\Work_Archive\\_Data_Raw\\ecoinvent\\3.3\\ei_33_rawdata_parsed.xls')           

In [4]:
FN = FileNames[2]
File = 'C:\\Users\\spauliuk\\FILES\\Work_Archive\\_Data_Raw\\ecoinvent\\3.3\\ei33_unlinked\\' + FN + '.spold'
e = xml.etree.ElementTree.parse(File).getroot()

In [35]:
#for child in e[0][1]:
#    print(child.tag, child.attrib)

{http://www.EcoInvent.org/EcoSpold02}intermediateExchange {'productionVolumeAmount': '4', 'casNumber': '10049-04-4', 'id': 'ceee6ba7-819c-40df-bb1f-dc9fac164173', 'intermediateExchangeId': '29fbcb1d-79ca-45df-8eec-3e51bb95979d', 'unitId': '487df68b-4994-4027-8fdc-a4dc298257b7', 'amount': '1'}
{http://www.EcoInvent.org/EcoSpold02}intermediateExchange {'casNumber': '7647-14-5', 'intermediateExchangeId': 'edcfa83a-363a-4b2d-8d6a-612e32dfcdfc', 'unitId': '487df68b-4994-4027-8fdc-a4dc298257b7', 'amount': '0.912', 'id': 'b8903fc8-7b98-4885-aadb-ecd990a7fe51'}
{http://www.EcoInvent.org/EcoSpold02}intermediateExchange {'intermediateExchangeId': '759b89bd-3aa6-42ad-b767-5bb9ef5d331d', 'unitId': '77ae64fa-7e74-4252-9c3b-889c1cd20bfc', 'amount': '0.333', 'id': 'a8f9baab-42bf-435e-a083-12177c584430'}
{http://www.EcoInvent.org/EcoSpold02}intermediateExchange {'intermediateExchangeId': '3d0fe4e0-eac9-4704-b3fd-09b8594d0fbe', 'unitId': '5b972631-34e3-4db7-a615-f6931770a0cb', 'amount': '4E-10', 'id': 

In [53]:
for child in e[0]:
    if child.tag == '{http://www.EcoInvent.org/EcoSpold02}activityDescription':
        for childx in child:
            print(childx)
            if childx.tag == '{http://www.EcoInvent.org/EcoSpold02}activity':
                for childxx in childx:
                    print(childxx)
                    if childxx.tag == '{http://www.EcoInvent.org/EcoSpold02}activityName':                    
                        print(childxx.text)
                    if childxx.tag == '{http://www.EcoInvent.org/EcoSpold02}generalComment':                    
                        for childxx3 in childxx:
                            if childxx3.attrib['index'] == '1':
                                print(childxx3.text)
            if childx.tag == '{http://www.EcoInvent.org/EcoSpold02}geography':
                for childxx2 in childx:
                    print(childxx2)
                    if childxx2.tag == '{http://www.EcoInvent.org/EcoSpold02}shortname':                    
                        print(childxx2.text)
            if childx.tag == '{http://www.EcoInvent.org/EcoSpold02}timePeriod':
                print(childx.attrib['startDate'])
                print(childx.attrib['endDate'])
                

<Element '{http://www.EcoInvent.org/EcoSpold02}activity' at 0x00000040A3945368>
<Element '{http://www.EcoInvent.org/EcoSpold02}activityName' at 0x00000040A3945958>
wood pellet production
<Element '{http://www.EcoInvent.org/EcoSpold02}synonym' at 0x00000040A3945EF8>
<Element '{http://www.EcoInvent.org/EcoSpold02}includedActivitiesStart' at 0x00000040A3945908>
<Element '{http://www.EcoInvent.org/EcoSpold02}includedActivitiesEnd' at 0x00000040A39455E8>
<Element '{http://www.EcoInvent.org/EcoSpold02}generalComment' at 0x00000040A39411D8>
This dataset represent the inputs and outputs of materials and energy for wood pellets production. Pellets are produced in a wood pellets factory which uses wood residue from sawmills and woodchips as raw materials. The raw materials are firstly pre-treated and dryed, then comminuted and mixed. In the end they are pellettized, cooled and bagged. Pellets that are packed in 15 kg-bags amount to the 20% of the production. The remaining 80% are sold unpacked. 

In [14]:
for child in e[0]:
    if child.tag == '{http://www.EcoInvent.org/EcoSpold02}activityDescription':
        for childx in child:
            print(childx)

<Element '{http://www.EcoInvent.org/EcoSpold02}activity' at 0x000000C4F1FAAC78>
<Element '{http://www.EcoInvent.org/EcoSpold02}classification' at 0x000000C4F1FAA818>
<Element '{http://www.EcoInvent.org/EcoSpold02}classification' at 0x000000C4F1FAA5E8>
<Element '{http://www.EcoInvent.org/EcoSpold02}geography' at 0x000000C4F1FAA7C8>
<Element '{http://www.EcoInvent.org/EcoSpold02}technology' at 0x000000C4F1FC20E8>
<Element '{http://www.EcoInvent.org/EcoSpold02}timePeriod' at 0x000000C4F1F834A8>
<Element '{http://www.EcoInvent.org/EcoSpold02}macroEconomicScenario' at 0x000000C4F1F83408>


In [16]:
for x in child:
    print(x.tag)


{http://www.EcoInvent.org/EcoSpold02}dataEntryBy
{http://www.EcoInvent.org/EcoSpold02}dataGeneratorAndPublication
{http://www.EcoInvent.org/EcoSpold02}fileAttributes


In [23]:
e[0][0][3]

<Element '{http://www.EcoInvent.org/EcoSpold02}geography' at 0x000000C4F1FAA7C8>